In [1]:
import os
import numpy as np
from sklearn.preprocessing import scale
import pickle as pkl

In [2]:
os.chdir('..')

with open('data/frequencies_wolfram.txt', 'r') as f:
    freqs = [i.strip() for i in f.readlines()]

freqs = freqs[1:]

with open('data/allowed_words.txt','r') as f:
    words = [i.strip() for i in f.readlines()]
    
word_freq_dict = {}
smallest = 1

FileNotFoundError: [Errno 2] No such file or directory: 'data/frequencies_wolfram.txt'

In [ ]:
# The frequencies are in a strange form that Mathematica gives, so we need to convert them to a form that can be used in Python
# For missing values I just used the smallest value.

for ind, value in enumerate(freqs):
    value = value.replace('*', '*10')
    value = value.replace('^', '**')
    try:
        value = eval(value)
        word_freq_dict[words[ind]] = value
        if value < smallest:
            smallest = value
    except:
        word_freq_dict[words[ind]] = -1


for key, value in word_freq_dict.items():
    if value == -1:
        word_freq_dict[key] = smallest
        
# Scaling seemed like a good idea, IDK.
values = np.array(list(word_freq_dict.values()))
values = scale(values)

In [ ]:
# After manipulating the numbers a bit we put them through a sigmoid function.
# So by manipulating i and j we set the steepness of the curve and the point where it will equal 0.5

# Once the steepness was decided, i ran this because we know that there are about 2500 possible answers to wordle and it felt like a good number to use.

i = 0
j = 500
while True:
    values0 = (values + i) * j
    s_values = 1/(1 + np.exp(-values0))
    sorted_list = sorted(s_values, reverse=True)
    if sorted_list[2500] > 0.5:
        print(i)
        break
    i += 0.001

# This was to manually check the probability scores of the words.
cor_tups = [(k,v) for k,v in zip(words, s_values)]

with open('data/frequencies_words_alpha.txt', 'w') as file:
    for i in cor_tups:
        file.write(i + '\n')

sorted_tups = sorted(cor_tups, key=lambda x: x[1], reverse=True)
with open('data/frequencies_words_sorted.txt', 'w') as file:
    for i in sorted_tups:
        file.write(i + '\n')
        

0.09200000000000007


In [ ]:
cor_dict = {k: v for k,v in zip(words, s_values)}

with open('data/word_probs.pkl', 'wb') as f:
    pkl.dump(cor_dict, f)
    
# Maybe a pickle is faster to load?
with open('data/allowed_words.pkl', 'wb') as f:
    pkl.dump(words, f)
    